In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

In [2]:
ships_csv = DATA_DIR / "global_cargo_ships.csv"   # Kaggle dump
df_ships = (
    pd.read_csv(ships_csv)
    .rename(columns={
        "Company_Name": "company_name",
        "ship_name": "ship_type",
    })
)
df_ships["age"] = pd.Timestamp.now().year - df_ships["built_year"]
df_ships = df_ships.drop(columns=["built_year"])

df_ships.head()

,company_name,ship_type,gt,dwt,length,width,age
0,PRELUDE,Offshore Support Vessel,499167,394330,489,74,8
1,MSC LORETO,Container Ship,236184,240000,399,60,2
2,MSC FEBE,Container Ship,232618,228149,400,62,6
3,MSC ARINA,Container Ship,228741,228111,400,61,6
4,EVER GOVERN,Container Ship,219688,198937,400,59,6


In [3]:
mrv_path = DATA_DIR / "eu_mrv_public.xlsx"

rename_map = {
    "IMO Number":                                   "imo",
    "Name":                                         "name",
    "Technical efficiency":                         "tech_efficiency",
    "Port of Registry":                             "port_of_registry",
    "Total fuel consumption [m tonnes]":            "fuel_consumption",
    "Total CO₂ emissions [m tonnes]":               "total_co2_emissions",
    "Annual Time spent at sea [hours]":             "annual_hrs_at_sea",
}

df_mrv = (
    pd.read_excel(
        mrv_path,
        sheet_name=0,
        engine="openpyxl",
        dtype=str,
        skiprows=2,
    )
    .rename(columns=rename_map)
    [list(rename_map.values())]
)

df_mrv[["tech_eff_index", "tech_eff_value"]] = df_mrv["tech_efficiency"].str.extract(r"([A-Za-z]+)\s*\(([\d\.]+)\s*gCO₂/t·nm\)")
df_mrv = df_mrv.drop(columns=["tech_efficiency"])

ship_name_word_sets = {
    name: set(str(name).lower().split())
    for name in df_ships["company_name"].dropna().unique()
}

mrv_name_word_sets = {
    name: set(str(name).lower().split())
    for name in df_mrv["name"].dropna().unique() # Use unique names
}

cols_from_mrv = [col for col in df_mrv.columns if col != 'name']

matched_rows_data = []

for ship_name, ship_words in ship_name_word_sets.items():
    ship_rows = df_ships.loc[df_ships["company_name"] == ship_name]
    for mrv_name, mrv_words in mrv_name_word_sets.items():
        if ship_words <= mrv_words or mrv_words <= ship_words:
            mrv_row_match = df_mrv.loc[df_mrv["name"] == mrv_name]

            if not mrv_row_match.empty:
                mrv_row = mrv_row_match.iloc[0]

                for _, ship_row in ship_rows.iterrows():
                    row_data = {}

                    for col in df_ships.columns:
                        row_data[col] = ship_row[col]

                    for col in cols_from_mrv:
                        if col not in row_data:
                             row_data[col] = mrv_row[col]


                    matched_rows_data.append(row_data)

df_final = pd.DataFrame(matched_rows_data)

df_final.head()


/Users/kbuloichyk/Documents/GreenBerth/.venv/lib/python3.13/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,company_name,ship_type,gt,dwt,length,width,age,imo,port_of_registry,fuel_consumption,total_co2_emissions,annual_hrs_at_sea,tech_eff_index,tech_eff_value
0,PRELUDE,Offshore Support Vessel,499167,394330,489,74,8,9365623,Port Victoria,2689,8474.77,2177.73,EIV,7.29
1,PRELUDE,Offshore Support Vessel,499167,394330,489,74,8,9810654,PANAMA,503.9,1583.55,593.4,EEDI,5.05
2,MSC LORETO,Container Ship,236184,240000,399,60,2,9934735,Monrovia,14308.65,44602.04,2668,EEXI,6.21
3,MSC FEBE,Container Ship,232618,228149,400,62,6,9839478,Panama,6075.67,18956.3,1199.23,EEXI,7.29
4,MSC ARINA,Container Ship,228741,228111,400,61,6,9839284,Panama,16857.12,52535.95,3088.9,EEXI,7.08


In [4]:
unctad_csv = DATA_DIR / "unctad_portcalls.csv"

rename_map = {
    "CommercialMarket_Label": "market_label",
    "Median_time_in_port_days_Value": "median_time_in_port_days",
}

df_un = (
    pd.read_csv(unctad_csv)
    .rename(columns=rename_map)
    [list(rename_map.values())]
)

market_label_to_ship_name_map = {
    "Liquid bulk carriers"              : "Crude Oil Tanker",
    "Liquefied natural gas carriers"    : "LNG Tanker",
    "Liquefied petroleum gas carriers"  : "LNG Tanker",
    "Dry bulk carriers"                 : "Bulk Carrier",
    "Dry breakbulk carriers"            : "Bulk Carrier",
    "Roll-on/ roll-off ships"           : "Container Ship",
    "Container ships"                   : "Container Ship",
    "Passenger ships"                   : "Passenger (Cruise) Ship",
    "All ships"                         : "Offshore Support Vessel",
}

df_un["ship_type"] = df_un["market_label"].map(market_label_to_ship_name_map)
df_un = df_un.drop(columns=["market_label"])

def get_median_time(idx, ship_type):
    matches = df_un[df_un["ship_type"] == ship_type]["median_time_in_port_days"].dropna().reset_index(drop=True)
    if not matches.empty:
        return matches.iloc[idx % len(matches)]
    return np.nan

df_final["median_time_in_port_days"] = [
    get_median_time(idx, ship_name) for idx, ship_name in enumerate(df_final["ship_type"])
]

df_final["median_time_in_port_hours"] = [
    float(get_median_time(idx, ship_name)) * 24 if pd.notna(get_median_time(idx, ship_name)) else np.nan
    for idx, ship_name in enumerate(df_final["ship_type"])
]

df_final.drop(columns=["median_time_in_port_days"], inplace=True)

df_final.head()

,company_name,ship_type,gt,dwt,length,width,age,imo,port_of_registry,fuel_consumption,total_co2_emissions,annual_hrs_at_sea,tech_eff_index,tech_eff_value,median_time_in_port_hours
0,PRELUDE,Offshore Support Vessel,499167,394330,489,74,8,9365623,Port Victoria,2689,8474.77,2177.73,EIV,7.29,23.76
1,PRELUDE,Offshore Support Vessel,499167,394330,489,74,8,9810654,PANAMA,503.9,1583.55,593.4,EEDI,5.05,36.96
2,MSC LORETO,Container Ship,236184,240000,399,60,2,9934735,Monrovia,14308.65,44602.04,2668,EEXI,6.21,35.04
3,MSC FEBE,Container Ship,232618,228149,400,62,6,9839478,Panama,6075.67,18956.3,1199.23,EEXI,7.29,14.40
4,MSC ARINA,Container Ship,228741,228111,400,61,6,9839284,Panama,16857.12,52535.95,3088.9,EEXI,7.08,16.08


In [5]:
df_final["imo"] = pd.to_numeric(df_final["imo"], errors="coerce", downcast="integer")
df_final["total_co2_emissions"] = pd.to_numeric(df_final["total_co2_emissions"], errors="coerce")
df_final["fuel_consumption"] = pd.to_numeric(df_final["fuel_consumption"], errors="coerce")
df_final["annual_hrs_at_sea"] = pd.to_numeric(df_final["annual_hrs_at_sea"], errors="coerce")
df_final["tech_eff_value"] = pd.to_numeric(df_final["tech_eff_value"], errors="coerce")

In [6]:
np.random.seed(42) # for reproducibility
df_final['annual_distance_sailed_placeholder'] = np.random.uniform(10000, 50000, size=len(df_final))

ship_type_mapping = {
    'Bulk Carrier': {'imo_type': 'Bulk Carrier', 'capacity_measure': 'dwt'},
    'Container Ship': {'imo_type': 'Container Ship', 'capacity_measure': 'dwt'},
    'Crude Oil Tanker': {'imo_type': 'Tanker', 'capacity_measure': 'dwt'},
    'LNG Tanker': {'imo_type': 'LNG Carrier', 'capacity_measure': 'dwt'},
    'Passenger (Cruise) Ship': {'imo_type': 'Cruise Passenger Ship', 'capacity_measure': 'gt'},
    'Offshore Support Vessel': {'imo_type': 'General Cargo Ship_Fallback', 'capacity_measure': 'dwt'},
}

imo_cii_parameters = {
    'Bulk Carrier': {'a': 4745, 'c': 0.622, 'dd': [0.86, 0.94, 1.06, 1.18]},
    'Container Ship': {'a': 1984, 'c': 0.489, 'dd': [0.83, 0.94, 1.07, 1.18]},
    'Tanker': {'a': 5247, 'c': 0.610, 'dd': [0.82, 0.93, 1.08, 1.28]},
    'LNG Carrier': {
        'low_dwt': {'a': 14479e10, 'c': 2.673, 'dd': [0.78, 0.92, 1.10, 1.37]},
        'high_dwt': {'a': 9.827, 'c': 0.000, 'dd': [0.89, 0.98, 1.06, 1.13]}
    },
    'Cruise Passenger Ship': {'a': 930, 'c': 0.383, 'dd': [0.87, 0.95, 1.06, 1.16]},
    'General Cargo Ship_Fallback': {
        'low_dwt': {'a': 588, 'c': 0.3885, 'dd': [0.76, 0.89, 1.08, 1.27]},
        'high_dwt': {'a': 31948, 'c': 0.792, 'dd': [0.76, 0.89, 1.08, 1.27]}
    }
}

reduction_factor_2023 = 0.05

def get_cii_parameters(imo_ship_type, dwt):
    if imo_ship_type == 'LNG Carrier':
        if dwt is not None and dwt >= 100000:
            return imo_cii_parameters[imo_ship_type]['high_dwt']
        elif dwt is not None:
             return imo_cii_parameters[imo_ship_type]['low_dwt']
        else:
            return None
    elif imo_ship_type == 'General Cargo Ship_Fallback':
        if dwt is not None and dwt >= 20000:
            return imo_cii_parameters[imo_ship_type]['high_dwt']
        elif dwt is not None:
            return imo_cii_parameters[imo_ship_type]['low_dwt']
        else:
             return None
    else:
        return imo_cii_parameters.get(imo_ship_type)

def calculate_cii_rating(row):
    user_ship_type = row['ship_type']

    imo_type_info = ship_type_mapping.get(user_ship_type)

    if not imo_type_info:
         return 'N/A - Unmapped Ship Type'

    imo_type = imo_type_info['imo_type']
    capacity_measure = imo_type_info['capacity_measure']

    capacity = row[capacity_measure]
    annual_distance_sailed = row['annual_distance_sailed_placeholder']
    total_co2_emissions_g = row['total_co2_emissions'] * 1000000

    if pd.isna(capacity) or capacity == 0 or pd.isna(annual_distance_sailed) or annual_distance_sailed == 0 or pd.isna(total_co2_emissions_g):
        return 'N/A - Missing Data'

    params = get_cii_parameters(imo_type, row['dwt'])

    if not params:
         return 'N/A - Parameter Retrieval Error'

    attained_cii = total_co2_emissions_g / (capacity * annual_distance_sailed)

    try:
        reference_cii = params['a'] * (capacity**-params['c'])
    except (TypeError, ValueError):
         return 'N/A - Reference CII Calculation Error'

    required_cii_2023 = (1 - reduction_factor_2023) * reference_cii

    if required_cii_2023 == 0:
        return 'N/A - Required CII is Zero'

    cii_ratio = attained_cii / required_cii_2023

    dd_vectors = params['dd']

    if not isinstance(dd_vectors, list) or len(dd_vectors) != 4:
         return 'N/A - Invalid dd vectors'

    if cii_ratio < dd_vectors[0]:
        rating = 'A'
    elif cii_ratio < dd_vectors[1]:
        rating = 'B'
    elif cii_ratio < dd_vectors[2]:
        rating = 'C'
    elif cii_ratio < dd_vectors[3]:
        rating = 'D'
    else:
        rating = 'E'

    return rating

df_final['cii_rating'] = df_final.apply(calculate_cii_rating, axis=1)

df_final = df_final.drop(columns=['annual_distance_sailed_placeholder'])

df_final["cii_rating"].value_counts()



cii_rating
A    132
E     31
C      9
D      8
B      4
Name: count, dtype: int64

In [7]:
df_final.columns.values

array(['company_name', 'ship_type', 'gt', 'dwt', 'length', 'width', 'age',
       'imo', 'port_of_registry', 'fuel_consumption',
       'total_co2_emissions', 'annual_hrs_at_sea', 'tech_eff_index',
       'tech_eff_value', 'median_time_in_port_hours', 'cii_rating'],
      dtype=object)

In [8]:
df_final.dropna(inplace=True) # fixme: better not to drop them
df_final.to_csv(DATA_DIR / "final_dataset.csv", index=False)